In [1]:

'''
    0. Initialise with cols list of lists (list of columns)
    COLUMN
    1. Create list for headers from Order field names
    2. Add to the list, with product header items
    3. create list for customer details
    4. On the same list index, append product counts for that customer
    5. Total row
    6. formatting
'''
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
# 0. Cols is to be the eventual list of lists
cols = []

In [4]:
from order.models import Order
ord = Order.objects.all().first()
ord

<Order: [NNotARealOrder] Christopher Koliba: RH4 2WJ by Delivery>